# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

In [9]:
# make a new table where each row from patent also has its citations, 
# and filter out rows where the patent does not have a postate
nq1 = patents.join(citations, [(patents.PATENT == citations.CITING), (patents.POSTATE.isNotNull())]).cache()

In [26]:
# join the new table created above back with patents to retrieve the
# postate of the citations, such that the postate of the citations matches
# the postate of their citing patent
nq1 = nq1.alias('nq1')
patents = patents.alias('patents')
nq2 = nq1.join(patents, [(col('nq1.CITED') == col('patents.PATENT')) & (col('nq1.POSTATE')==col('patents.POSTATE'))]).cache()

In [69]:
# group the rows by the citing patents and count the number of rows 
# for each unique patent
nq3 = nq2.groupBy(col('nq1.PATENT')).count().cache()
nq3 = nq3.alias('nq3')

In [80]:
# reformat the columns to match the required output and sort the rows
# in decreasing order of cocitation counts
nq3.join(patents, [(col('nq3.PATENT')==col('patents.PATENT'))]).sort(col('nq3.count').desc()).select([col('patents.PATENT'), 
                                                                                                                      col('patents.GYEAR'), 
                                                                                                                      col('patents.GDATE'), 
                                                                                                                      col('patents.APPYEAR'), 
                                                                                                                      col('patents.COUNTRY'), 
                                                                                                                      col('patents.POSTATE'), 
                                                                                                                      col('patents.ASSIGNEE'), 
                                                                                                                      col('patents.ASSCODE'), 
                                                                                                                      col('patents.CLAIMS'), 
                                                                                                                      col('patents.NCLASS'), 
                                                                                                                      col('patents.CAT'),
                                                                                                                      col('patents.SUBCAT'), 
                                                                                                                      col('patents.CMADE'), 
                                                                                                                      col('patents.CRECEIVE'), 
                                                                                                                      col('patents.RATIOCIT'), 
                                                                                                                      col('patents.GENERAL'), 
                                                                                                                      col('patents.ORIGINAL'), 
                                                                                                                      col('patents.FWDAPLAG'), 
                                                                                                                      col('patents.BCKGTLAG'), 
                                                                                                                      col('patents.SELFCTUB'), 
                                                                                                                      col('patents.SELFCTLB'), 
                                                                                                                      col('patents.SECDUPBD'), 
                                                                                                                      col('patents.SECDLWBD'),
                                                                                                                      col('count').alias('SAME_STATE')]).show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 